In [6]:

import os
import warnings
from argparse import ArgumentParser
import matplotlib.patches as patches
import torch
from tqdm import tqdm
import cv2
from torch.utils import data
# 개별 json 라벨 파일을 이용해 학습 데이터 리스트 생성
from glob import glob
import xml.etree.ElementTree as ET
from xml.dom import minidom
import os
from nets import nn
from utils import util
from utils.dataset import Dataset
from torch.utils import data
import numpy as np
import matplotlib.pyplot as plt
import random
import openslide
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:",device)
params={'names':{
  0: 'pd-l1 negative tumor cell',
  1: 'pd-l1 positive tumor cell',
  2: 'non-tumor cell'}}

# params={'names':{
#   0: 'pd-l1 negative tumor cell',
#   1: 'pd-l1 positive tumor cell'}}

device: cuda:0


In [7]:
save_dir='../../model/yolov11/'
model = nn.yolo_v11_m(len(params['names'])).to(device)
checkpoint_path = os.path.join(save_dir, 'best_model.pt')
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path, map_location=device,weights_only=False)
    model.load_state_dict(checkpoint['model_state_dict'])
    
def pred_patch(torch_patch, model, start_x, start_y,magnification):
    model.eval()
    conf_threshold=0.15
    iou_threshold=0.45
    prediction_count = 0
    negative_tumor=[]
    positive_tumor=[]
    non_tumor=[]
    with torch.no_grad():
        img_input = torch_patch
        with torch.amp.autocast('cuda'):
            pred = model(img_input)
        
        # NMS 적용
        results = util.non_max_suppression(pred, confidence_threshold=conf_threshold, iou_threshold=iou_threshold)
        
        if len(results[0]) > 0:
            for *xyxy, conf, cls_id in results[0]:
                x1, y1, x2, y2 = xyxy
                x1, y1, x2, y2 = x1.item(), y1.item(), x2.item(), y2.item()
                w_pred = x2 - x1
                h_pred = y2 - y1
                center_x = (x1 + x2)//2
                center_y = (y1 + y2)//2
                if cls_id.item() == 0: #pd-l1 negative tumor cell
                    negative_tumor.append({'x':start_x+center_x*magnification,'y':start_y+center_y*magnification,'cls_id':0})
                elif cls_id.item() == 1: #pd-l1 positive tumor cell
                    positive_tumor.append({'x':start_x+center_x*magnification,'y':start_y+center_y*magnification,'cls_id':1})
                else: #non-tumor cell
                    non_tumor.append({'x':start_x+center_x*magnification,'y':start_y+center_y*magnification,'cls_id':2})

                
                prediction_count += 1
    return negative_tumor, positive_tumor, non_tumor

In [ ]:
slide_path=glob('../../data/WSI_test/*.ndpi')
image_size=512 # 모델 입력 크기
origin_mpp=0.25
output_mpp=0.5
original_size=int(image_size*output_mpp/origin_mpp) #1122
magnification=original_size/image_size
count=0
i=0
file_name=os.path.basename(slide_path[i]).split('.')[0]
slide=openslide.OpenSlide(slide_path[i])
thumbnail=slide.get_thumbnail((slide.dimensions[0]//64, slide.dimensions[1]//64))
thumb_mask=cv2.threshold(255-np.array(thumbnail.convert('L')),30,255,cv2.THRESH_BINARY)[1]
thumb_mask=cv2.morphologyEx(thumb_mask,cv2.MORPH_CLOSE,np.ones((15,15),np.uint8))
thumb_mask=cv2.morphologyEx(thumb_mask,cv2.MORPH_OPEN,np.ones((5,5),np.uint8))
patch_row=0
negative_tumor=[]
positive_tumor=[]
non_tumor=[]
for patch_row in tqdm(range(slide.dimensions[0]//original_size-1)):
    for patch_col in range(slide.dimensions[1]//original_size-1):
        if np.sum(thumb_mask[(patch_col*original_size)//64:((patch_col+1)*original_size)//6,(patch_row*original_size)//64:((patch_row+1)*original_size)//64])>0:
            count+=1
            patch=slide.read_region((patch_row*original_size, patch_col*original_size), level=0, size=(original_size, original_size)).resize((image_size,image_size))
            torch_patch=torch.from_numpy(np.array(patch)[:,:,:3]).permute(2,0,1).unsqueeze(0).float()/255.
            torch_patch=torch_patch.to(device)
            temp_negative_tumor, temp_positive_tumor, temp_non_tumor = pred_patch(torch_patch, model, patch_row*original_size, patch_col*original_size, magnification)
            negative_tumor.extend(temp_negative_tumor)
            positive_tumor.extend(temp_positive_tumor)
            non_tumor.extend(temp_non_tumor)
print('Total patch count:',count)

In [ ]:
def create_asap_xml(negative_tumor, positive_tumor, non_tumor, output_path):
    """
    세포 검출 결과를 ASAP XML 형식으로 저장하는 함수
    
    Args:
        negative_tumor: PD-L1 negative tumor cells 리스트
        positive_tumor: PD-L1 positive tumor cells 리스트  
        non_tumor: Non-tumor cells 리스트
        output_path: 저장할 XML 파일 경로
    """
    
    # 루트 엘리먼트 생성
    root = ET.Element("ASAP_Annotations")
    
    # Annotations 엘리먼트 생성
    annotations = ET.SubElement(root, "Annotations")
    
    annotation_id = 0
    
    # Negative tumor cells 추가 (빨간색)
    for cell in negative_tumor:
        annotation = ET.SubElement(annotations, "Annotation")
        annotation.set("Name", f"Annotation {annotation_id}")
        annotation.set("Type", "Dot")
        annotation.set("PartOfGroup", "Negative Tumor")
        annotation.set("Color", "#FF0000")  # 빨간색
        
        coordinates = ET.SubElement(annotation, "Coordinates")
        coordinate = ET.SubElement(coordinates, "Coordinate")
        coordinate.set("Order", "0")
        coordinate.set("X", str(float(cell['x'])))
        coordinate.set("Y", str(float(cell['y'])))
        
        annotation_id += 1
    
    # Positive tumor cells 추가 (파란색)
    for cell in positive_tumor:
        annotation = ET.SubElement(annotations, "Annotation")
        annotation.set("Name", f"Annotation {annotation_id}")
        annotation.set("Type", "Dot")
        annotation.set("PartOfGroup", "Positive Tumor")
        annotation.set("Color", "#0000FF")  # 파란색
        
        coordinates = ET.SubElement(annotation, "Coordinates")
        coordinate = ET.SubElement(coordinates, "Coordinate")
        coordinate.set("Order", "0")
        coordinate.set("X", str(float(cell['x'])))
        coordinate.set("Y", str(float(cell['y'])))
        
        annotation_id += 1
    
    # Non-tumor cells 추가 (녹색)
    # for cell in non_tumor:
    #     annotation = ET.SubElement(annotations, "Annotation")
    #     annotation.set("Name", f"Annotation {annotation_id}")
    #     annotation.set("Type", "Dot")
    #     annotation.set("PartOfGroup", "Non Tumor")
    #     annotation.set("Color", "#00FF00")  # 녹색
        
    #     coordinates = ET.SubElement(annotation, "Coordinates")
    #     coordinate = ET.SubElement(coordinates, "Coordinate")
    #     coordinate.set("Order", "0")
    #     coordinate.set("X", str(float(cell['x'])))
    #     coordinate.set("Y", str(float(cell['y'])))
        
    #     annotation_id += 1
    
    # AnnotationGroups 엘리먼트 생성
    annotation_groups = ET.SubElement(root, "AnnotationGroups")
    
    # Negative Tumor 그룹
    group1 = ET.SubElement(annotation_groups, "Group")
    group1.set("Name", "Negative Tumor")
    group1.set("PartOfGroup", "None")
    group1.set("Color", "#0000FF")
    attributes1 = ET.SubElement(group1, "Attributes")
    
    # Positive Tumor 그룹
    group2 = ET.SubElement(annotation_groups, "Group")
    group2.set("Name", "Positive Tumor")
    group2.set("PartOfGroup", "None")
    group2.set("Color", "#FF0000")
    attributes2 = ET.SubElement(group2, "Attributes")
    
    # Non Tumor 그룹
    # group3 = ET.SubElement(annotation_groups, "Group")
    # group3.set("Name", "Non Tumor")
    # group3.set("PartOfGroup", "None")
    # group3.set("Color", "#00FF00")
    # attributes3 = ET.SubElement(group3, "Attributes")
    
    # XML을 예쁘게 포맷팅하여 저장
    rough_string = ET.tostring(root, 'unicode')
    reparsed = minidom.parseString(rough_string)
    pretty_xml = reparsed.toprettyxml(indent="	")
    
    # <?xml version... 라인을 원하는 형태로 수정
    lines = pretty_xml.split('\n')
    lines[0] = '<?xml version="1.0"?>'
    pretty_xml = '\n'.join(lines[1:])  # 빈 라인 제거
    
    with open(output_path, 'w', encoding='utf-8') as f:
        f.write(pretty_xml)
    
    print(f"XML 파일이 저장되었습니다: {output_path}")
    print(f"총 검출된 세포 수:")
    print(f"  - Negative tumor cells: {len(negative_tumor)}개")
    print(f"  - Positive tumor cells: {len(positive_tumor)}개") 
    print(f"  - Non-tumor cells: {len(non_tumor)}개")
    print(f"  - 전체: {len(negative_tumor) + len(positive_tumor) + len(non_tumor)}개")

# XML 파일 생성
output_xml_path = f"../../results/{file_name}_predictions.xml"
os.makedirs("../../results", exist_ok=True)

create_asap_xml(negative_tumor, positive_tumor, non_tumor, output_xml_path)